<a href="https://colab.research.google.com/github/Annie00000/Project/blob/main/2_19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### file 下載

1. 使用 html.Button：取代超連結，當按鈕被點擊時，觸發下載。
2. 使用 dcc.Download：讓 dcc.Download 控制檔案下載行為。
3. 新增 callback：
 - 監聽點擊的按鈕，判斷是哪個檔案需要下載。
 - 讀取對應的檔案並觸發 dcc.send_file() 下載。
4.確保 file_id 唯一：使用 month/{file} 確保每個檔案按鈕都有獨特 ID。

In [ ]:
import dash
from dash import dcc, html, dash_table, Input, Output
import pandas as pd
import os

app = dash.Dash(__name__)

# 原始數據
data = [
    [1, "H02U", 'DH240P23.00', 23, '202502_A.txt, 202502_B.xlsx'],
    [2, "H02D", 'DH240P11.00', 23, '202502_C.png, 202502_D.csv'],
    [3, "H03F", 'DH240P10.00', 21, '202502_CD.xlsx, 202502_E.csv, 2025-02_F.csv'],
    [4, "D03F", 'CH240P10.00', 11, '']
]

# 轉換為 DataFrame
df = pd.DataFrame(data, columns=["Issue Number", "Chart Name", "Lot", "Count", "File"])

def generate_file_links(file_string):
    if not file_string:
        return ""  # 空值處理

    files = file_string.split(', ')
    links = []

    for file in files:
        parts = file.split('_', 1)
        if len(parts) > 1:
            display_name = parts[1]  # 取 _ 之後的字眼
            month = parts[0]  # 取 _ 之前的月份字串
            file_id = f"{month}/{file}"  # 構造唯一識別符
            link = html.Button(display_name, id={'type': 'download-btn', 'index': file_id}, n_clicks=0)
            links.append(link)

    return links

df["File"] = df["File"].apply(generate_file_links)

app.layout = html.Div([
    dash_table.DataTable(
        id='table',
        columns=[
            {"name": col, "id": col} if col != "File" else {"name": col, "id": col, "presentation": "markdown"}
            for col in df.columns
        ],
        data=df.to_dict('records'),
        style_table={'overflowX': 'auto'},
    ),
    dcc.Download(id="download-component")
])

@app.callback(
    Output("download-component", "data"),
    Input({'type': 'download-btn', 'index': dash.ALL}, "n_clicks"),
    prevent_initial_call=True
)
def trigger_download(n_clicks):
    ctx = dash.callback_context
    if not ctx.triggered:
        return dash.no_update

    triggered_id = eval(ctx.triggered[0]["prop_id"].split(".")[0])  # 獲取觸發的 ID
    file_path = f"../../data_source/{triggered_id}"  # 構造檔案路徑

    if os.path.exists(file_path):
        return dcc.send_file(file_path)

    return dash.no_update

if __name__ == '__main__':
    app.run_server(debug=True)


按鈕是 html.Button，預設外觀是普通按鈕，希望它看起來像超連結（藍色+底線），可以

1. 透過 html.A 加 dcc.Link，或者調整 html.Button 的 CSS 樣式。

In [ ]:
link = html.A(display_name, href="#", id={'type': 'download-btn', 'index': file_id}, style={"color": "blue", "textDecoration": "underline", "cursor": "pointer"})

2. 修改 html.Button 樣式

In [ ]:
link = html.Button(display_name, id={'type': 'download-btn', 'index': file_id}, n_clicks=0, style={"background": "none", "border": "none", "color": "blue", "textDecoration": "underline", "cursor": "pointer"})
